In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy qiskit-ibm-catalog qiskit-serverless

# Transpilationsoptimierungen mit SABRE
*Geschätzte Nutzungsdauer: unter einer Minute auf einem Heron-r2-Prozessor (HINWEIS: Dies ist nur eine Schätzung. Deine tatsächliche Laufzeit kann abweichen.)*
## Hintergrund
Transpilation ist ein entscheidender Schritt in Qiskit, der Quantenschaltkreise in Formen umwandelt, die mit spezifischer Quantenhardware kompatibel sind. Sie umfasst zwei wesentliche Phasen: **Qubit-Layout** (Zuordnung logischer Qubits zu physischen Qubits auf dem Gerät) und **Gate-Routing** (Sicherstellung, dass Multi-Qubit-Gates die Gerätekonnektivität einhalten, indem bei Bedarf SWAP-Gates eingefügt werden).

SABRE (*SWAP-Based Bidirectional heuristic search algorithm*) ist ein leistungsstarkes Optimierungswerkzeug für Layout und Routing. Es ist besonders effektiv für **großskalige Schaltkreise** (100+ Qubits) und Geräte mit komplexen Kopplungskarten, wie den **IBM&reg; Heron**, bei denen das exponentielle Wachstum möglicher Qubit-Zuordnungen effiziente Lösungen erfordert.

### Warum SABRE verwenden?
SABRE minimiert die Anzahl der SWAP-Gates und reduziert die Schaltkreistiefe, wodurch die Leistung des Schaltkreises auf realer Hardware verbessert wird. Sein heuristikbasierter Ansatz macht es ideal für fortschrittliche Hardware und große, komplexe Schaltkreise. Kürzlich eingeführte Verbesserungen im [LightSABRE](https://arxiv.org/abs/2409.08368)-Algorithmus optimieren die Leistung von SABRE weiter und bieten schnellere Laufzeiten sowie weniger SWAP-Gates. Diese Verbesserungen machen es noch effektiver für großskalige Schaltkreise.

### Was du lernen wirst
Dieses Tutorial ist in zwei Teile gegliedert:
1. Lerne, SABRE mit **Qiskit-Patterns** für die fortgeschrittene Optimierung großer Schaltkreise zu verwenden.
2. Nutze **qiskit_serverless**, um das Potenzial von SABRE für skalierbare und effiziente Transpilation voll auszuschöpfen.

Du wirst:
- SABRE für Schaltkreise mit 100+ Qubits optimieren und dabei die Standard-Transpilationseinstellungen wie `optimization_level=3` übertreffen.
- **LightSABRE-Verbesserungen** erkunden, die die Laufzeit verbessern und die Gate-Anzahl reduzieren.
- Wichtige SABRE-Parameter (`swap_trials`, `layout_trials`, `max_iterations`, `heuristic`) anpassen, um **Schaltkreisqualität** und **Transpilationslaufzeit** auszubalancieren.
## Voraussetzungen
Bevor du mit diesem Tutorial beginnst, stelle sicher, dass Folgendes installiert ist:
- Qiskit SDK v1.0 oder höher, mit [Visualisierungs](https://docs.quantum.ibm.com/api/qiskit/visualization)-Unterstützung
- Qiskit Runtime v0.28 oder höher (`pip install qiskit-ibm-runtime`)
- Serverless (`pip install qiskit-ibm-catalog qiskit_serverless`)
## Einrichtung

In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_catalog import QiskitServerless, QiskitFunction
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorOptions
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import matplotlib.pyplot as plt
import numpy as np
import time

## Teil I. Verwendung von SABRE mit Qiskit-Patterns

SABRE kann in Qiskit verwendet werden, um Quantenschaltkreise zu optimieren, indem es sowohl das Qubit-Layout als auch die Gate-Routing-Phase behandelt. In diesem Abschnitt führen wir dich durch das **Minimalbeispiel** der Verwendung von SABRE mit Qiskit-Patterns, wobei der Schwerpunkt auf Schritt 2 der Optimierung liegt.

Um SABRE auszuführen, benötigst du:
- Eine **DAG**-Darstellung (Directed Acyclic Graph) deines Quantenschaltkreises.
- Die **Kopplungskarte** des Backends, die angibt, wie Qubits physisch verbunden sind.
- Den **SABRE-Pass**, der den Algorithmus zur Optimierung von Layout und Routing anwendet.

In diesem Teil konzentrieren wir uns auf den **SabreLayout**-Pass. Er führt sowohl Layout- als auch Routing-Versuche durch und arbeitet daran, das effizienteste anfängliche Layout zu finden und gleichzeitig die Anzahl der benötigten SWAP-Gates zu minimieren. Wichtig ist, dass `SabreLayout` allein intern sowohl das Layout als auch das Routing optimiert, indem es die Lösung speichert, die die geringste Anzahl von SWAP-Gates hinzufügt. Beachte, dass bei alleiniger Verwendung von **SabreLayout** die Heuristik von SABRE nicht geändert werden kann, aber die Anzahl der `layout_trials` angepasst werden kann.

### Schritt 1: Klassische Eingaben auf ein Quantenproblem abbilden

Ein **GHZ-Schaltkreis (Greenberger-Horne-Zeilinger)** ist ein Quantenschaltkreis, der einen verschränkten Zustand vorbereitet, in dem sich alle Qubits entweder im Zustand `|0...0⟩` oder `|1...1⟩` befinden. Der GHZ-Zustand für $n$ Qubits wird mathematisch dargestellt als:
$$ |\text{GHZ}\rangle = \frac{1}{\sqrt{2}} \left( |0\rangle^{\otimes n} + |1\rangle^{\otimes n} \right) $$

Er wird konstruiert durch:
1. Ein Hadamard-Gate auf das erste Qubit, um Superposition zu erzeugen.
2. Eine Reihe von CNOT-Gates, um die übrigen Qubits mit dem ersten zu verschränken.

Für dieses Beispiel konstruieren wir absichtlich einen **GHZ-Schaltkreis mit Stern-Topologie** anstelle einer linearen Topologie. In der Stern-Topologie fungiert das erste Qubit als „Hub", und alle anderen Qubits werden direkt über CNOT-Gates mit ihm verschränkt. Diese Wahl ist bewusst, denn während der **GHZ-Zustand mit linearer Topologie** theoretisch in $ O(N) $ Tiefe auf einer linearen Kopplungskarte ohne SWAP-Gates implementiert werden kann, würde SABRE trivialerweise eine optimale Lösung finden, indem es einen 100-Qubit-GHZ-Schaltkreis auf einen Teilgraphen der Heavy-Hex-Kopplungskarte des Backends abbildet.

Der **GHZ-Schaltkreis mit Stern-Topologie** stellt ein deutlich anspruchsvolleres Problem dar. Obwohl er theoretisch ebenfalls in $ O(N) $ Tiefe ohne SWAP-Gates ausgeführt werden kann, erfordert das Finden dieser Lösung die Identifikation eines optimalen anfänglichen Layouts, was aufgrund der nichtlinearen Konnektivität des Schaltkreises wesentlich schwieriger ist. Diese Topologie dient als besserer Testfall für die Bewertung von SABRE, da sie zeigt, wie Konfigurationsparameter die Layout- und Routing-Leistung unter komplexeren Bedingungen beeinflussen.

![ghz_star_topology.png](../docs/images/tutorials/transpilation-optimizations-with-sabre/ghz_star_topology.avif)

Bemerkenswert:
- Das **HighLevelSynthesis**-Werkzeug kann die optimale $ O(N) $-Tiefenlösung für den GHZ-Schaltkreis mit Stern-Topologie ohne Einführung von SWAP-Gates erzeugen, wie im obigen Bild gezeigt.
- Alternativ kann der **StarPrerouting**-Pass die Tiefe weiter reduzieren, indem er die Routing-Entscheidungen von SABRE leitet, obwohl er möglicherweise dennoch einige SWAP-Gates einführt. StarPrerouting erhöht jedoch die Laufzeit und erfordert die Integration in den anfänglichen Transpilationsprozess.

Für die Zwecke dieses Tutorials schließen wir sowohl HighLevelSynthesis als auch StarPrerouting aus, um den direkten Einfluss der SABRE-Konfiguration auf Laufzeit und Schaltkreistiefe zu isolieren und hervorzuheben. Durch die Messung des Erwartungswerts $ \langle Z_0 Z_i \rangle $ für jedes Qubit-Paar analysieren wir:
- Wie gut SABRE SWAP-Gates und Schaltkreistiefe reduziert.
- Den Effekt dieser Optimierungen auf die Wiedergabetreue des ausgeführten Schaltkreises, wobei Abweichungen von $ \langle Z_0 Z_i \rangle = 1 $ einen Verlust der Verschränkung anzeigen.!

In [2]:
# set seed for reproducibility
seed = 42
num_qubits = 110

# Create GHZ circuit
qc = QuantumCircuit(num_qubits)
qc.h(0)
for i in range(1, num_qubits):
    qc.cx(0, i)

qc.measure_all()

Als Nächstes bilden wir die relevanten Operatoren ab, um das Verhalten des Systems zu bewerten. Konkret verwenden wir `ZZ`-Operatoren zwischen Qubits, um zu untersuchen, wie die Verschränkung mit zunehmender Entfernung der Qubits abnimmt. Diese Analyse ist entscheidend, da Ungenauigkeiten in den Erwartungswerten $\langle Z_0 Z_i \rangle$ für entfernte Qubits den Einfluss von Rauschen und Fehlern bei der Schaltkreisausführung offenbaren können. Durch die Untersuchung dieser Abweichungen gewinnen wir Einblick, wie gut der Schaltkreis die Verschränkung unter verschiedenen SABRE-Konfigurationen bewahrt und wie effektiv SABRE die Auswirkungen von Hardware-Einschränkungen minimiert.

In [3]:
# ZZII...II, ZIZI...II, ... , ZIII...IZ
operator_strings = [
    "Z" + "I" * i + "Z" + "I" * (num_qubits - 2 - i)
    for i in range(num_qubits - 1)
]
print(operator_strings)
print(len(operator_strings))

operators = [SparsePauliOp(operator) for operator in operator_strings]

['ZZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

### Step 2: Optimize problem for quantum hardware execution

In this step, we focus on optimizing the circuit layout for execution on a specific quantum hardware device with 127 qubits. This is the main focus of the tutorial, as we perform **SABRE optimizations and transpilation** to achieve the best circuit performance. Using the `SabreLayout` pass, we determine an initial qubit mapping that minimizes the need for SWAP gates during routing. By passing the `coupling_map` of the target backend, `SabreLayout` adapts the layout to the device's connectivity constraints.

We will use `generate_preset_pass_manager` with `optimization_level=3` for the transpilation process and customize the `SabreLayout` pass with different configurations. The goal is to find a setup that produces a transpiled circuit with the **lowest size and/or depth**, demonstrating the impact of SABRE optimizations.

#### Why Are Circuit Size and Depth Important?

- **Lower size (gate count):** Reduces the number of operations, minimizing opportunities for errors to accumulate.
- **Lower depth:** Shortens the overall execution time, which is critical for avoiding decoherence and maintaining quantum state fidelity.

By optimizing these metrics, we improve the circuit’s reliability and execution accuracy on noisy quantum hardware.

Select the backend.

In [4]:
service = QiskitRuntimeService()
# backend = service.least_busy(
#    operational=True, simulator=False, min_num_qubits=127
# )
backend = service.backend("ibm_boston")
print(f"Using backend: {backend.name}")

Using backend: ibm_boston


### Schritt 2: Problem für die Ausführung auf Quantenhardware optimieren
In diesem Schritt konzentrieren wir uns auf die Optimierung des Schaltkreis-Layouts für die Ausführung auf einem bestimmten Quantenhardware-Gerät mit 127 Qubits. Dies ist der Schwerpunkt des Tutorials, da wir **SABRE-Optimierungen und Transpilation** durchführen, um die bestmögliche Schaltkreisleistung zu erzielen. Mit dem `SabreLayout`-Pass bestimmen wir eine anfängliche Qubit-Zuordnung, die den Bedarf an SWAP-Gates während des Routings minimiert. Durch Übergabe der `coupling_map` des Ziel-Backends passt `SabreLayout` das Layout an die Konnektivitätsbeschränkungen des Geräts an.

Wir verwenden `generate_preset_pass_manager` mit `optimization_level=3` für den Transpilationsprozess und passen den `SabreLayout`-Pass mit verschiedenen Konfigurationen an. Ziel ist es, eine Konfiguration zu finden, die einen transpilierten Schaltkreis mit der **niedrigsten Größe und/oder Tiefe** erzeugt und so den Einfluss der SABRE-Optimierungen demonstriert.

#### Warum sind Schaltkreisgröße und -tiefe wichtig?
- **Geringere Größe (Gate-Anzahl):** Reduziert die Anzahl der Operationen und minimiert die Möglichkeiten für Fehlerakkumulation.
- **Geringere Tiefe:** Verkürzt die gesamte Ausführungszeit, was entscheidend ist, um Dekohärenz zu vermeiden und die Wiedergabetreue des Quantenzustands zu erhalten.

Durch die Optimierung dieser Metriken verbessern wir die Zuverlässigkeit und Ausführungsgenauigkeit des Schaltkreises auf verrauschter Quantenhardware.
Wähle das Backend aus.

In [5]:
# Get the coupling map from the backend
cmap = CouplingMap(backend().configuration().coupling_map)

# Create the SabreLayout passes for the custom configurations
sl_2 = SabreLayout(
    coupling_map=cmap,
    seed=seed,
    max_iterations=4,
    layout_trials=200,
    swap_trials=200,
)
sl_3 = SabreLayout(
    coupling_map=cmap,
    seed=seed,
    max_iterations=8,
    layout_trials=200,
    swap_trials=200,
)

# Create the pass managers, need to first create then configure the SabreLayout passes
pm_1 = generate_preset_pass_manager(
    optimization_level=3, backend=backend, seed_transpiler=seed
)
pm_2 = generate_preset_pass_manager(
    optimization_level=3, backend=backend, seed_transpiler=seed
)
pm_3 = generate_preset_pass_manager(
    optimization_level=3, backend=backend, seed_transpiler=seed
)

Now we can configure the `SabreLayout` pass in the custom pass managers. To do this we know that for the default `generate_preset_pass_manager` on `optimization_level=3`, the `SabreLayout` pass is at index 2, as `SabreLayout` occurs after `SetLayout` and `VF2Laout` passes. We can access this pass and modify its parameters.

In [6]:
pm_2.layout.replace(index=2, passes=sl_2)
pm_3.layout.replace(index=2, passes=sl_3)

Um den Einfluss verschiedener Konfigurationen auf die Schaltkreisoptimierung zu bewerten, erstellen wir drei Pass-Manager mit jeweils unterschiedlichen Einstellungen für den `SabreLayout`-Pass. Diese Konfigurationen helfen dabei, den Kompromiss zwischen Schaltkreisqualität und Transpilationszeit zu analysieren.

#### Wichtige Parameter
- **`max_iterations`**: Die Anzahl der Vorwärts-Rückwärts-Routing-Iterationen zur Verfeinerung des Layouts und Reduzierung der Routing-Kosten.
- **`layout_trials`**: Die Anzahl der getesteten zufälligen Anfangslayouts, wobei dasjenige ausgewählt wird, das die SWAP-Gates minimiert.
- **`swap_trials`**: Die Anzahl der Routing-Versuche für jedes Layout, um die Gate-Platzierung für besseres Routing zu verfeinern.

Erhöhe `layout_trials` und `swap_trials`, um eine gründlichere Optimierung durchzuführen, allerdings auf Kosten einer erhöhten Transpilationszeit.

#### Konfigurationen in diesem Tutorial
1. **`pm_1`**: Standardeinstellungen mit `optimization_level=3`.
   - `max_iterations=4`
   - `layout_trials=20`
   - `swap_trials=20`

2. **`pm_2`**: Erhöht die Anzahl der Versuche für eine bessere Exploration.
   - `max_iterations=4`
   - `layout_trials=200`
   - `swap_trials=200`

3. **`pm_3`**: Erweitert `pm_2` durch Erhöhung der Iterationsanzahl für weitere Verfeinerung.
   - `max_iterations=8`
   - `layout_trials=200`
   - `swap_trials=200`

Durch den Vergleich der Ergebnisse dieser Konfigurationen wollen wir bestimmen, welche das beste Gleichgewicht zwischen Schaltkreisqualität (zum Beispiel Größe und Tiefe) und Rechenaufwand erzielt.

In [7]:
# Transpile the circuit with each pass manager and measure the time
t0 = time.time()
tqc_1 = pm_1.run(qc)
t1 = time.time() - t0
t0 = time.time()
tqc_2 = pm_2.run(qc)
t2 = time.time() - t0
t0 = time.time()
tqc_3 = pm_3.run(qc)
t3 = time.time() - t0

# Obtain the depths and the total number of gates (circuit size)
depth_1 = tqc_1.depth(lambda x: x.operation.num_qubits == 2)
depth_2 = tqc_2.depth(lambda x: x.operation.num_qubits == 2)
depth_3 = tqc_3.depth(lambda x: x.operation.num_qubits == 2)
size_1 = tqc_1.size()
size_2 = tqc_2.size()
size_3 = tqc_3.size()

# Transform the observables to match the backend's ISA
operators_list_1 = [op.apply_layout(tqc_1.layout) for op in operators]
operators_list_2 = [op.apply_layout(tqc_2.layout) for op in operators]
operators_list_3 = [op.apply_layout(tqc_3.layout) for op in operators]

# Compute improvements compared to pass manager 1 (default)
depth_improvement_2 = ((depth_1 - depth_2) / depth_1) * 100
depth_improvement_3 = ((depth_1 - depth_3) / depth_1) * 100
size_improvement_2 = ((size_1 - size_2) / size_1) * 100
size_improvement_3 = ((size_1 - size_3) / size_1) * 100
time_increase_2 = ((t2 - t1) / t1) * 100
time_increase_3 = ((t3 - t1) / t1) * 100

print(
    f"Pass manager 1 (4,20,20)  : Depth {depth_1}, Size {size_1}, Time {t1:.4f} s"
)
print(
    f"Pass manager 2 (4,200,200): Depth {depth_2}, Size {size_2}, Time {t2:.4f} s"
)
print(f"  - Depth improvement: {depth_improvement_2:.2f}%")
print(f"  - Size improvement: {size_improvement_2:.2f}%")
print(f"  - Time increase: {time_increase_2:.2f}%")
print(
    f"Pass manager 3 (8,200,200): Depth {depth_3}, Size {size_3}, Time {t3:.4f} s"
)
print(f"  - Depth improvement: {depth_improvement_3:.2f}%")
print(f"  - Size improvement: {size_improvement_3:.2f}%")
print(f"  - Time increase: {time_increase_3:.2f}%")

Pass manager 1 (4,20,20)  : Depth 439, Size 2346, Time 0.5775 s
Pass manager 2 (4,200,200): Depth 395, Size 2070, Time 3.9927 s
  - Depth improvement: 10.02%
  - Size improvement: 11.76%
  - Time increase: 591.43%
Pass manager 3 (8,200,200): Depth 375, Size 1873, Time 2.3079 s
  - Depth improvement: 14.58%
  - Size improvement: 20.16%
  - Time increase: 299.67%


Nun können wir den `SabreLayout`-Pass in den benutzerdefinierten Pass-Managern konfigurieren. Dazu wissen wir, dass beim Standard-`generate_preset_pass_manager` mit `optimization_level=3` der `SabreLayout`-Pass an Index 2 steht, da `SabreLayout` nach den Passes `SetLayout` und `VF2Laout` kommt. Wir können auf diesen Pass zugreifen und seine Parameter ändern.

In [8]:
# Plot the results of the metrics
times = [t1, t2, t3]
depths = [depth_1, depth_2, depth_3]
sizes = [size_1, size_2, size_3]
pm_names = [
    "pm_1 (4 iter, 20 trials)",
    "pm_2 (4 iter, 200 trials)",
    "pm_3 (8 iter, 200 trials)",
]
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(pm_names)))

# Create a figure with three subplots
fig, axs = plt.subplots(3, 1, figsize=(6, 9), sharex=True)
axs[0].bar(pm_names, times, color=colors)
axs[0].set_ylabel("Time (s)", fontsize=12)
axs[0].set_title("Transpilation Time", fontsize=14)
axs[0].grid(axis="y", linestyle="--", alpha=0.7)
axs[1].bar(pm_names, depths, color=colors)
axs[1].set_ylabel("Depth", fontsize=12)
axs[1].set_title("Circuit Depth", fontsize=14)
axs[1].grid(axis="y", linestyle="--", alpha=0.7)
axs[2].bar(pm_names, sizes, color=colors)
axs[2].set_ylabel("Size", fontsize=12)
axs[2].set_title("Circuit Size", fontsize=14)
axs[2].set_xticks(range(len(pm_names)))
axs[2].set_xticklabels(pm_names, fontsize=10, rotation=15)
axs[2].grid(axis="y", linestyle="--", alpha=0.7)

# Add some spacing between subplots
plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/transpilation-optimizations-with-sabre/extracted-outputs/818a8997-d2c7-4661-a6ea-f58eac376bf8-0.avif" alt="Output of the previous code cell" />

Nachdem jeder Pass-Manager konfiguriert ist, führen wir nun den Transpilationsprozess für jeden aus. Um die Ergebnisse zu vergleichen, verfolgen wir wichtige Metriken, darunter die Transpilationszeit, die Tiefe des Schaltkreises (gemessen als Zwei-Qubit-Gate-Tiefe) und die Gesamtzahl der Gates in den transpilierten Schaltkreisen.

In [9]:
options = EstimatorOptions()
options.resilience_level = 2
options.dynamical_decoupling.enable = True
options.dynamical_decoupling.sequence_type = "XY4"

# Create an Estimator object
estimator = Estimator(backend, options=options)

In [10]:
# Submit the circuit to Estimator
job_1 = estimator.run([(tqc_1, operators_list_1)])
job_1_id = job_1.job_id()
print(job_1_id)

job_2 = estimator.run([(tqc_2, operators_list_2)])
job_2_id = job_2.job_id()
print(job_2_id)

job_3 = estimator.run([(tqc_3, operators_list_3)])
job_3_id = job_3.job_id()
print(job_3_id)

d5k0qs7853es738dab6g
d5k0qsf853es738dab70
d5k0qsf853es738dab7g


In [11]:
# Run the jobs
result_1 = job_1.result()[0]
print("Job 1 done")
result_2 = job_2.result()[0]
print("Job 2 done")
result_3 = job_3.result()[0]
print("Job 3 done")

Job 1 done
Job 2 done
Job 3 done


![Output of the previous code cell](../docs/images/tutorials/transpilation-optimizations-with-sabre/extracted-outputs/818a8997-d2c7-4661-a6ea-f58eac376bf8-0.avif)

### Schritt 3: Ausführung mit Qiskit-Primitiven
In diesem Schritt verwenden wir das `Estimator`-Primitiv, um die Erwartungswerte $\langle Z_0 Z_i \rangle$ für die `ZZ`-Operatoren zu berechnen und die Verschränkung sowie die Ausführungsqualität der transpilierten Schaltkreise zu bewerten. Um typische Benutzer-Workflows abzubilden, reichen wir den Job zur Ausführung ein und wenden Fehlerunterdrückung mittels **Dynamischer Entkopplung** an, einer Technik, die Dekohärenz abschwächt, indem Gate-Sequenzen zur Erhaltung der Qubit-Zustände eingefügt werden. Zusätzlich geben wir eine Resilienzstufe an, um Rauschen entgegenzuwirken, wobei höhere Stufen genauere Ergebnisse auf Kosten einer erhöhten Verarbeitungszeit liefern. Dieser Ansatz bewertet die Leistung jeder Pass-Manager-Konfiguration unter realistischen Ausführungsbedingungen.

In [12]:
data = list(range(1, len(operators) + 1))  # Distance between the Z operators

values_1 = list(result_1.data.evs)
values_2 = list(result_2.data.evs)
values_3 = list(result_3.data.evs)

plt.plot(
    data,
    values_1,
    marker="o",
    label="pm_1 (iters=4, swap_trials=20, layout_trials=20)",
)
plt.plot(
    data,
    values_2,
    marker="s",
    label="pm_2 (iters=4, swap_trials=200, layout_trials=200)",
)
plt.plot(
    data,
    values_3,
    marker="^",
    label="pm_3 (iters=8, swap_trials=200, layout_trials=200)",
)
plt.xlabel("Distance between qubits $i$")
plt.ylabel(r"$\langle Z_i Z_0 \rangle / \langle Z_1 Z_0 \rangle $")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/transpilation-optimizations-with-sabre/extracted-outputs/bc6cb36f-4bf2-4275-baf5-9557fcba520a-0.avif" alt="Output of the previous code cell" />

### Analysis of Results

The plot shows the expectation values $\langle Z_0 Z_i \rangle / \langle Z_0 Z_0 \rangle$  as a function of the distance between qubits for three pass manager configurations with increasing levels of optimization. In the ideal case, these values remain close to 1, indicating strong correlations across the circuit. As the distance increases, noise and accumulated errors lead to a decay in correlations, revealing how well each transpilation strategy preserves the underlying structure of the state.

Among the three configurations, `pm_1` clearly performs the worst. Its correlation values decay rapidly as the distance increases and approach zero much earlier than the other two configurations. This behavior is consistent with its larger circuit depth and gate count, where accumulated noise quickly degrades long-range correlations.

Both `pm_2` and `pm_3` represent significant improvements over `pm_1` across essentially all distances. On average, `pm_3` exhibits the strongest overall performance, maintaining higher correlation values over longer distances and showing a more gradual decay. This aligns with its more aggressive optimization, which produces shallower circuits that are generally more robust to noise accumulation.

That said, `pm_2` shows noticeably better accuracy at short distances compared to `pm_3`, despite having a slightly larger depth and gate count. This suggests that circuit depth alone does not fully determine performance; the specific structure produced by the transpilation, including how entangling gates are arranged and how errors propagate through the circuit, also plays an important role. In some cases, the transformations applied by `pm_2` appear to better preserve local correlations, even if they do not scale as well to longer distances.

Taken together, these results highlight a trade-off between circuit compactness and circuit structure. While increased optimization generally improves long-range stability, the best performance for a given observable depends on both reducing circuit depth and producing a structure that is well matched to the noise characteristics of the hardware.

## Part II. Configuring the heuristic in SABRE and using Serverless

In addition to adjusting trial numbers, SABRE supports customization of the routing heuristic used during transpilation. By default, `SabreLayout` employs the decay heuristic, which dynamically weights qubits based on their likelihood of being swapped. To use a different heuristic (such as the `lookahead` heuristic), you can create a custom `SabreSwap` pass and connect it to `SabreLayout` by running a `PassManager` with `FullAncillaAllocation`, `EnlargeWithAncilla`, and `ApplyLayout`. When using `SabreSwap` as a parameter for `SabreLayout`, only one layout trial is performed by default. To efficiently run multiple layout trials, we leverage the serverless runtime for parallelization. For more about serverless, see the [Serverless documentation](/docs/guides/serverless).

### How to Change the Routing Heuristic
1. Create a custom `SabreSwap` pass with the desired heuristic.
2. Use this custom `SabreSwap` as the routing method for the `SabreLayout` pass.

While it is possible to run multiple layout trials using a loop, serverless runtime is the better choice for large-scale and more vigorous experiments. Serverless supports parallel execution of layout trials, significantly speeding up the optimization of larger circuits and large experimental sweeps. This makes it especially valuable when working with resource-intensive tasks or when time efficiency is critical.

This section focuses solely on step 2 of optimization: minimizing circuit size and depth to achieve the best possible transpiled circuit. Building on the earlier results, we now explore how heuristic customization and serverless parallelization can further enhance optimization performance, making it suitable for large-scale quantum circuit transpilation.

### Results without serverless runtime (1 layout trial):

In [17]:
swap_trials = 1000

# Default PassManager with `SabreLayout` and `SabreSwap`, using heuristic "decay"
sr_default = SabreSwap(
    coupling_map=cmap, heuristic="decay", trials=swap_trials, seed=seed
)
sl_default = SabreLayout(
    coupling_map=cmap, routing_pass=sr_default, seed=seed
)
pm_default = generate_preset_pass_manager(
    optimization_level=3, backend=backend, seed_transpiler=seed
)
pm_default.layout.replace(index=2, passes=sl_default)
pm_default.routing.replace(index=1, passes=sr_default)

t0 = time.time()
tqc_default = pm_default.run(qc)
t_default = time.time() - t0
size_default = tqc_default.size()
depth_default = tqc_default.depth(lambda x: x.operation.num_qubits == 2)


# Custom PassManager with `SabreLayout` and `SabreSwap`, using heuristic "lookahead"
sr_custom = SabreSwap(
    coupling_map=cmap, heuristic="lookahead", trials=swap_trials, seed=seed
)
sl_custom = SabreLayout(coupling_map=cmap, routing_pass=sr_custom, seed=seed)
pm_custom = generate_preset_pass_manager(
    optimization_level=3, backend=backend, seed_transpiler=seed
)
pm_custom.layout.replace(index=2, passes=sl_custom)
pm_custom.routing.replace(index=1, passes=sr_custom)

t0 = time.time()
tqc_custom = pm_custom.run(qc)
t_custom = time.time() - t0
size_custom = tqc_custom.size()
depth_custom = tqc_custom.depth(lambda x: x.operation.num_qubits == 2)

print(
    f"Default (heuristic='decay')    : Depth {depth_default}, Size {size_default}, Time {t_default}"
)
print(
    f"Custom  (heuristic='lookahead'): Depth {depth_custom}, Size {size_custom}, Time {t_custom}"
)

Default (heuristic='decay')    : Depth 443, Size 3115, Time 1.034372091293335
Custom  (heuristic='lookahead'): Depth 432, Size 2856, Time 0.6669301986694336


Here we see that the `lookahead` heuristic performs better than the `decay` heuristic in terms of circuit depth, size, and time. This improvements highlights how we can improve SABRE beyond just trials and iterations for your specific circuit and hardware constraints. Note that these results are based on a single layout trial. To achieve more accurate results, we recommend running multiple layout trials, which can be done efficiently using the serverless runtime.

### Results with serverless runtime (multiple layout trials)

Qiskit Serverless requires setting up your workload’s `.py` files into a dedicated directory. The following code cell is a Python file in the `source_files` directory named `transpile_remote.py`. This file contains the function that runs the transpilation process.

In [18]:
# This cell is hidden from users, it makes sure the `source_files` directory exists
from pathlib import Path

Path("source_files").mkdir(exist_ok=True)

In [26]:
%%writefile source_files/transpile_remote.py
import time
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler import CouplingMap
from qiskit_serverless import get_arguments, save_result, distribute_task, get
from qiskit_ibm_runtime import QiskitRuntimeService

@distribute_task(target={
    "cpu": 1,
    "mem": 1024 * 1024 * 1024
})
def transpile_remote(qc, optimization_level, backend_name, seed, swap_trials, heuristic):
    """Transpiles an abstract circuit into an ISA circuit for a given backend."""

    service = QiskitRuntimeService()
    backend = service.backend(backend_name)

    pm = generate_preset_pass_manager(
        optimization_level=optimization_level,
        backend=backend,
        seed_transpiler=seed
    )

    # Changing the `SabreLayout` and `SabreSwap` passes to use the custom configurations
    cmap = CouplingMap(backend().configuration().coupling_map)
    sr = SabreSwap(coupling_map=cmap, heuristic=heuristic, trials=swap_trials, seed=seed)
    sl = SabreLayout(coupling_map=cmap, routing_pass=sr, seed=seed)
    pm.layout.replace(index=2, passes=sl)
    pm.routing.replace(index=1, passes=sr)

    # Measure the transpile time
    start_time = time.time()  # Start timer
    tqc = pm.run(qc)  # Transpile the circuit
    end_time = time.time()  # End timer

    transpile_time = end_time - start_time  # Calculate the elapsed time
    return tqc, transpile_time  # Return both the transpiled circuit and the transpile time


# Get program arguments
arguments = get_arguments()
circuit = arguments.get("circuit")
backend_name = arguments.get("backend_name")
optimization_level = arguments.get("optimization_level")
seed_list = arguments.get("seed_list")
swap_trials = arguments.get("swap_trials")
heuristic = arguments.get("heuristic")

# Transpile the circuits
transpile_worker_references = [
    transpile_remote(circuit, optimization_level, backend_name, seed, swap_trials, heuristic)
    for seed in seed_list
]

results_with_times = get(transpile_worker_references)

# Separate the transpiled circuits and their transpile times
transpiled_circuits = [result[0] for result in results_with_times]
transpile_times = [result[1] for result in results_with_times]

# Save both results and transpile times
save_result({"transpiled_circuits": transpiled_circuits, "transpile_times": transpile_times})

Overwriting source_files/transpile_remote.py


The following cell uploads the `transpile_remote.py` file as a Qiskit Serverless program under the name `transpile_remote_serverless`.

In [27]:
serverless = QiskitServerless()

transpile_remote_demo = QiskitFunction(
    title="transpile_remote_serverless",
    entrypoint="transpile_remote.py",
    working_dir="./source_files/",
)
serverless.upload(transpile_remote_demo)
transpile_remote_serverless = serverless.load("transpile_remote_serverless")

### Schritt 4: Nachbearbeitung und Rückgabe der Ergebnisse im gewünschten klassischen Format
Sobald der Job abgeschlossen ist, analysieren wir die Ergebnisse, indem wir die Erwartungswerte $\langle Z_0 Z_i \rangle$ für jedes Qubit darstellen. In einer idealen Simulation sollten alle $\langle Z_0 Z_i \rangle$-Werte gleich 1 sein, was perfekte Verschränkung über alle Qubits hinweg widerspiegelt. Aufgrund von Rauschen und Hardware-Einschränkungen nehmen die Erwartungswerte jedoch typischerweise mit zunehmendem `i` ab, was zeigt, wie die Verschränkung mit der Entfernung abnimmt.

In diesem Schritt vergleichen wir die Ergebnisse jeder Pass-Manager-Konfiguration mit der idealen Simulation. Durch die Untersuchung der Abweichung von $\langle Z_0 Z_i \rangle$ von 1 für jede Konfiguration können wir quantifizieren, wie gut jeder Pass-Manager die Verschränkung bewahrt und die Auswirkungen von Rauschen abschwächt. Diese Analyse bewertet direkt den Einfluss der SABRE-Optimierungen auf die Ausführungstreue und hebt hervor, welche Konfiguration Optimierungsqualität und Ausführungsleistung am besten ausbalanciert.

Die Ergebnisse werden visualisiert, um Unterschiede zwischen den Pass-Managern hervorzuheben und zu zeigen, wie Verbesserungen bei Layout und Routing die endgültige Schaltkreisausführung auf verrauschter Quantenhardware beeinflussen.

In [28]:
num_seeds = 20  # represents the different layout trials
seed_list = [seed + i for i in range(num_seeds)]

![Output of the previous code cell](../docs/images/tutorials/transpilation-optimizations-with-sabre/extracted-outputs/bc6cb36f-4bf2-4275-baf5-9557fcba520a-0.avif)

### Analyse der Ergebnisse
Das Diagramm zeigt die Erwartungswerte $\langle Z_0 Z_i \rangle / \langle Z_0 Z_0 \rangle$ als Funktion der Entfernung zwischen Qubits für drei Pass-Manager-Konfigurationen mit zunehmenden Optimierungsstufen. Im Idealfall bleiben diese Werte nahe bei 1, was starke Korrelationen über den gesamten Schaltkreis hinweg anzeigt. Mit zunehmender Entfernung führen Rauschen und akkumulierte Fehler zu einem Abfall der Korrelationen, was zeigt, wie gut jede Transpilationsstrategie die zugrunde liegende Struktur des Zustands bewahrt.

Unter den drei Konfigurationen schneidet `pm_1` deutlich am schlechtesten ab. Seine Korrelationswerte fallen mit zunehmender Entfernung schnell ab und nähern sich viel früher als bei den beiden anderen Konfigurationen dem Nullwert. Dieses Verhalten stimmt mit seiner größeren Schaltkreistiefe und Gate-Anzahl überein, bei denen akkumuliertes Rauschen langreichweitige Korrelationen schnell verschlechtert.

Sowohl `pm_2` als auch `pm_3` stellen signifikante Verbesserungen gegenüber `pm_1` über im Wesentlichen alle Entfernungen dar. Im Durchschnitt zeigt `pm_3` die stärkste Gesamtleistung und bewahrt höhere Korrelationswerte über längere Entfernungen bei einem allmählicheren Abfall. Dies steht im Einklang mit seiner aggressiveren Optimierung, die flachere Schaltkreise erzeugt, die im Allgemeinen robuster gegenüber Rauschakkumulation sind.

Allerdings zeigt `pm_2` bei kurzen Entfernungen eine deutlich bessere Genauigkeit im Vergleich zu `pm_3`, obwohl es eine etwas größere Tiefe und Gate-Anzahl aufweist. Dies deutet darauf hin, dass die Schaltkreistiefe allein die Leistung nicht vollständig bestimmt; die spezifische Struktur, die durch die Transpilation erzeugt wird -- einschließlich der Anordnung verschränkender Gates und der Fehlerausbreitung durch den Schaltkreis -- spielt ebenfalls eine wichtige Rolle. In einigen Fällen scheinen die von `pm_2` angewandten Transformationen lokale Korrelationen besser zu erhalten, auch wenn sie bei längeren Entfernungen nicht so gut skalieren.


## Teil II. Konfiguration der Heuristik in SABRE und Verwendung von Serverless
Neben der Anpassung der Anzahl der Versuche unterstützt SABRE die Anpassung der Routing-Heuristik, die während der Transpilation verwendet wird. Standardmäßig verwendet `SabreLayout` die Decay-Heuristik, die Qubits dynamisch basierend auf ihrer Wahrscheinlichkeit, getauscht zu werden, gewichtet. Um eine andere Heuristik (wie die `lookahead`-Heuristik) zu verwenden, kannst du einen benutzerdefinierten `SabreSwap`-Pass erstellen und ihn mit `SabreLayout` verbinden, indem du einen `PassManager` mit `FullAncillaAllocation`, `EnlargeWithAncilla` und `ApplyLayout` ausführst. Wenn `SabreSwap` als Parameter für `SabreLayout` verwendet wird, wird standardmäßig nur ein Layout-Versuch durchgeführt. Um mehrere Layout-Versuche effizient auszuführen, nutzen wir die Serverless-Laufzeitumgebung zur Parallelisierung. Weitere Informationen zu Serverless findest du in der [Serverless-Dokumentation](/guides/serverless).

### So änderst du die Routing-Heuristik
1. Erstelle einen benutzerdefinierten `SabreSwap`-Pass mit der gewünschten Heuristik.
2. Verwende diesen benutzerdefinierten `SabreSwap` als Routing-Methode für den `SabreLayout`-Pass.

Obwohl es möglich ist, mehrere Layout-Versuche mithilfe einer Schleife durchzuführen, ist die Serverless-Laufzeitumgebung die bessere Wahl für große und umfangreichere Experimente. Serverless unterstützt die parallele Ausführung von Layout-Versuchen und beschleunigt die Optimierung größerer Schaltkreise und umfangreicher experimenteller Durchläufe erheblich. Dies macht es besonders wertvoll bei ressourcenintensiven Aufgaben oder wenn Zeiteffizienz entscheidend ist.

Dieser Abschnitt konzentriert sich ausschließlich auf Schritt 2 der Optimierung: die Minimierung der Schaltkreisgröße und -tiefe, um den bestmöglichen transpilierten Schaltkreis zu erzielen. Aufbauend auf den früheren Ergebnissen untersuchen wir nun, wie die Anpassung der Heuristik und die serverlose Parallelisierung die Optimierungsleistung weiter verbessern können, um sie für die Transpilation von Quantenschaltkreisen im großen Maßstab geeignet zu machen.
### Ergebnisse ohne Serverless-Laufzeitumgebung (1 Layout-Versuch):

In [29]:
job_lookahead = transpile_remote_serverless.run(
    circuit=qc,
    backend_name=backend.name,
    optimization_level=3,
    seed_list=seed_list,
    swap_trials=swap_trials,
    heuristic="lookahead",
)

In [30]:
job_lookahead.job_id

'15767dfc-e71d-4720-94d6-9212f72334c2'

In [31]:
job_lookahead.status()

'QUEUED'

Receive the logs and results from the serverless runtime.

In [21]:
logs_lookahead = job_lookahead.logs()
print(logs_lookahead)

No logs yet.


Once a program is `DONE`, you can use `job.results()` to fetch the result stored in `save_result()`.

In [32]:
# Run the job with lookahead heuristic
start_time = time.time()
results_lookahead = job_lookahead.result()
end_time = time.time()

job_lookahead_time = end_time - start_time

Die folgende Zelle lädt die Datei `transpile_remote.py` als Qiskit-Serverless-Programm unter dem Namen `transpile_remote_serverless` hoch.

In [33]:
job_decay = transpile_remote_serverless.run(
    circuit=qc,
    backend_name=backend.name,
    optimization_level=3,
    seed_list=seed_list,
    swap_trials=swap_trials,
    heuristic="decay",
)

In [34]:
job_decay.job_id

'00418c76-d6ec-4bd8-9f70-05d0fa14d4eb'

In [35]:
logs_decay = job_decay.logs()
print(logs_decay)

No logs yet.


In [36]:
# Run the job with the decay heuristic
start_time = time.time()
results_decay = job_decay.result()
end_time = time.time()

job_decay_time = end_time - start_time

In [37]:
# Extract transpilation times
transpile_times_decay = results_decay["transpile_times"]
transpile_times_lookahead = results_lookahead["transpile_times"]

# Calculate total transpilation time for serial execution
total_transpile_time_decay = sum(transpile_times_decay)
total_transpile_time_lookahead = sum(transpile_times_lookahead)

# Print total transpilation time
print("=== Total Transpilation Time (Serial Execution) ===")
print(f"Decay Heuristic    : {total_transpile_time_decay:.2f} seconds")
print(f"Lookahead Heuristic: {total_transpile_time_lookahead:.2f} seconds")

# Print serverless job time (parallel execution)
print("\n=== Serverless Job Time (Parallel Execution) ===")
print(f"Decay Heuristic    : {job_decay_time:.2f} seconds")
print(f"Lookahead Heuristic: {job_lookahead_time:.2f} seconds")

# Calculate and print average runtime per transpilation
avg_transpile_time_decay = total_transpile_time_decay / num_seeds
avg_transpile_time_lookahead = total_transpile_time_lookahead / num_seeds
avg_job_time_decay = job_decay_time / num_seeds
avg_job_time_lookahead = job_lookahead_time / num_seeds

print("\n=== Average Time Per Transpilation ===")
print(f"Decay Heuristic (Serial)    : {avg_transpile_time_decay:.2f} seconds")
print(f"Decay Heuristic (Serverless): {avg_job_time_decay:.2f} seconds")
print(
    f"Lookahead Heuristic (Serial)    : {avg_transpile_time_lookahead:.2f} seconds"
)
print(
    f"Lookahead Heuristic (Serverless): {avg_job_time_lookahead:.2f} seconds"
)

# Calculate and print serverless improvement percentage
decay_improvement_percentage = (
    (total_transpile_time_decay - job_decay_time) / total_transpile_time_decay
) * 100
lookahead_improvement_percentage = (
    (total_transpile_time_lookahead - job_lookahead_time)
    / total_transpile_time_lookahead
) * 100

print("\n=== Serverless Improvement ===")
print(f"Decay Heuristic    : {decay_improvement_percentage:.2f}%")
print(f"Lookahead Heuristic: {lookahead_improvement_percentage:.2f}%")

=== Total Transpilation Time (Serial Execution) ===
Decay Heuristic    : 112.37 seconds
Lookahead Heuristic: 85.37 seconds

=== Serverless Job Time (Parallel Execution) ===
Decay Heuristic    : 5.72 seconds
Lookahead Heuristic: 5.85 seconds

=== Average Time Per Transpilation ===
Decay Heuristic (Serial)    : 5.62 seconds
Decay Heuristic (Serverless): 0.29 seconds
Lookahead Heuristic (Serial)    : 4.27 seconds
Lookahead Heuristic (Serverless): 0.29 seconds

=== Serverless Improvement ===
Decay Heuristic    : 94.91%
Lookahead Heuristic: 93.14%


These results demonstrate the substantial efficiency gains from using serverless execution for quantum circuit transpilation. Compared to serial execution, serverless execution dramatically reduces overall runtime for both the `decay` and `lookahead` heuristics by parallelizing independent transpilation trials. While serial execution reflects the full cumulative cost of exploring multiple layout trials, the serverless job times highlight how parallel execution collapses this cost into a much shorter wall-clock time. As a result, the effective time per transpilation is reduced to a small fraction of that required in the serial setting, largely independent of the heuristic used. This capability is particularly important for optimizing SABRE to its fullest potential. Many of SABRE’s strongest performance gains come from increasing the number of layout and routing trials, which can be prohibitively expensive when executed sequentially. Serverless execution removes this bottleneck, enabling large-scale parameter sweeps and deeper exploration of heuristic configurations with minimal overhead.

Overall, these findings show that serverless execution is key to scaling SABRE optimization, making aggressive experimentation and refinement practical compared to serial execution.

Obtain the results from the serverless runtime and compare the results of the lookahead and decay heuristics. We will compare the sizes and depths.

In [38]:
# Extract sizes and depths
sizes_lookahead = [
    circuit.size() for circuit in results_lookahead["transpiled_circuits"]
]
depths_lookahead = [
    circuit.depth(lambda x: x.operation.num_qubits == 2)
    for circuit in results_lookahead["transpiled_circuits"]
]
sizes_decay = [
    circuit.size() for circuit in results_decay["transpiled_circuits"]
]
depths_decay = [
    circuit.depth(lambda x: x.operation.num_qubits == 2)
    for circuit in results_decay["transpiled_circuits"]
]


def create_scatterplot(x, y1, y2, xlabel, ylabel, title, labels, colors):
    plt.figure(figsize=(8, 5))
    plt.scatter(
        x, y1, label=labels[0], color=colors[0], alpha=0.8, edgecolor="k"
    )
    plt.scatter(
        x, y2, label=labels[1], color=colors[1], alpha=0.8, edgecolor="k"
    )
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.title(title, fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()


create_scatterplot(
    seed_list,
    sizes_lookahead,
    sizes_decay,
    "Seed",
    "Size",
    "Circuit Size",
    ["lookahead", "Decay"],
    ["blue", "red"],
)
create_scatterplot(
    seed_list,
    depths_lookahead,
    depths_decay,
    "Seed",
    "Depth",
    "Circuit Depth",
    ["lookahead", "Decay"],
    ["blue", "red"],
)

<Image src="../docs/images/tutorials/transpilation-optimizations-with-sabre/extracted-outputs/4cf9588b-8ea6-4761-b544-14bef8f0be85-0.avif" alt="Output of the previous code cell" />

<Image src="../docs/images/tutorials/transpilation-optimizations-with-sabre/extracted-outputs/4cf9588b-8ea6-4761-b544-14bef8f0be85-1.avif" alt="Output of the previous code cell" />

Each point in the scatter plots above represents a layout trial, with the x-axis indicating the circuit depth and the y-axis indicating the circuit size. The results reveal that the lookahead heuristic generally outperforms the decay heuristic in minimizing circuit depth and circuit size. In practical applications, the goal is to identify the optimal layout trial for your chosen heuristic, whether prioritizing depth or size. This can be achieved by selecting the trial with the lowest value for the desired metric. Importantly, increasing the number of layout trials improves the chances of achieving a better result in terms of size or depth, but it comes at the cost of higher computational overhead.

In [39]:
min_depth_lookahead = min(depths_lookahead)
min_depth_decay = min(depths_decay)
min_size_lookahead = min(sizes_lookahead)
min_size_decay = min(sizes_decay)
print(
    "Lookahead: Min Depth",
    min_depth_lookahead,
    "Min Size",
    min_size_lookahead,
)
print("Decay:     Min Depth", min_depth_decay, "Min Size", min_size_decay)

Lookahead: Min Depth 399 Min Size 2452
Decay:     Min Depth 415 Min Size 2611


Empfange die Protokolle und Ergebnisse von der Serverless-Laufzeitumgebung.

In [40]:
# This cell is hidden from users, it cleans up the `source_files` directory
from pathlib import Path

Path("source_files/transpile_remote.py").unlink()
Path("source_files").rmdir()

## Conclusion

In this tutorial, we explored how to optimize large circuits using SABRE in Qiskit. We demonstrated how to configure the `SabreLayout` pass with different parameters to balance circuit quality and transpilation runtime. We also showed how to customize the routing heuristic in SABRE and use the `QiskitServerless`runtime to parallelize layout trials efficiently for when `SabreSwap` is involved. By adjusting these parameters and heuristics, you can optimize the layout and routing of large circuits, ensuring they are executed efficiently on quantum hardware.

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_d9YWUSQIAvU9HXE)